In [2]:
import os
import numpy as np
import pandas as pd
import rasterio
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor

SEQ_LEN = 6                 
HORIZONS = 3               
PATCH_SIZE = 13             
HALF = PATCH_SIZE // 2
FILL_NAN_VALUE = 0.0

REQUIRED_COLS = [
    "era5_t2m_file", "era5_d2m_file", "era5_tp_file",
    "era5_u10_file", "era5_v10_file",
    "viirs_file", "dem_file", "lulc_file"
]

In [3]:
import tensorflow as tf
print(tf.__version__)

2.20.0


In [4]:
def _load_single_raster(path):
    with rasterio.open(path) as src:
        arr = src.read() 

    if arr.shape[0] == 1:
        
        return arr[0]
    else:
     
        return arr


In [5]:
def load_rasters(df, raster_cols, max_workers=8):
    
    all_paths = set()

    for col in raster_cols:
        if col in df.columns:
            all_paths.update(df[col].dropna().unique())
    all_paths = list(all_paths)

    cache = {}
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        results = list(ex.map(_load_single_raster, all_paths))

    for path, arr in zip(all_paths, results):
        if arr is not None:
            cache[path] = arr
    return cache


In [6]:
def _safe_center(h, w, patch_size=PATCH_SIZE):
    half = patch_size // 2
    r = np.clip(h // 2, half, h - half - 1)
    c = np.clip(w // 2, half, w - half - 1)
    return r, c

In [7]:
def _extract_patch(arr, row, col, patch_size=PATCH_SIZE):
    half = patch_size // 2
    h, w = arr.shape

    r0 = row - half
    r1 = row + half + 1
    c0 = col - half
    c1 = col + half + 1

    patch = np.zeros((patch_size, patch_size), dtype=arr.dtype)

    r0_clip = max(r0, 0)
    r1_clip = min(r1, h)
    c0_clip = max(c0, 0)
    c1_clip = min(c1, w)

    pr0 = r0_clip - r0
    pr1 = pr0 + (r1_clip - r0_clip)
    pc0 = c0_clip - c0
    pc1 = pc0 + (c1_clip - c0_clip)

    patch[pr0:pr1, pc0:pc1] = arr[r0_clip:r1_clip, c0_clip:c1_clip]

    return patch

In [8]:
def build_sample(seq_rows, horizon_rows, cache, force_fire=False):
    seq_patches = []

   
    for _, row in seq_rows.iterrows():
        bands = []
        for var in ["era5_t2m_file", "era5_d2m_file", "era5_tp_file",
                    "era5_u10_file", "era5_v10_file"]:
            arr = cache[row[var]]

            if len(arr.shape) == 3:
                arr = arr[0]

            h, w = arr.shape
            r, c = _safe_center(h, w)
            bands.append(_extract_patch(arr, r, c))

        dem = cache[row["dem_file"]]
        lulc = cache[row["lulc_file"]]

        if len(dem.shape) == 3:
            dem = dem[0]
        if len(lulc.shape) == 3:
            lulc = lulc[0]

        h, w = dem.shape
        r, c = _safe_center(h, w)
        bands.append(_extract_patch(dem, r, c))
        bands.append(_extract_patch(lulc, r, c))

        seq_patches.append(np.stack(bands, axis=-1))

    X = np.stack(seq_patches, axis=0)


    horizon_patches = []
   
    for _, row in horizon_rows.iterrows():
        viirs_stack = cache[row["viirs_file"]]
        
    
        target_band_idx_list = eval(row["target_band_idxs"])
 
        idx = target_band_idx_list[0]
        
        band = viirs_stack[idx - 1]
        h, w = band.shape
        r, c = _safe_center(h, w)

        if force_fire and np.any(band > 0):
            fire_pos = np.argwhere(band > 0)
            r, c = fire_pos[np.random.randint(len(fire_pos))]

        horizon_patches.append(_extract_patch(band, r, c))

    y = np.stack(horizon_patches, axis=0)

    return X.astype("float32"), y.astype("float32")

In [9]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf

# def make_generator(df, cache, fire_ratio=0.5):
    
#     valid_start_indices = list(range(len(df) - SEQ_LEN - HORIZONS + 1))

#     fire_start_indices = []
#     non_fire_start_indices = []
    
#     for i in valid_start_indices:
#         horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
#         # This line iterates through DataFrame rows to check for fire, which is fine.
#         has_fire = any(np.any(cache[row["viirs_file"]] > 0) for _, row in horizon_rows.iterrows())
        
#         if has_fire:
#             fire_start_indices.append(i)
#         else:
#             non_fire_start_indices.append(i)

#     # --- Start of sampling logic (your core strategy) ---
#     num_fire_samples = len(fire_start_indices)
    
#     if num_fire_samples == 0:
#         print("Warning: No fire events found in the dataset. Training will be difficult.")
#         # If no fires, take a small sample of non-fire indices
#         num_non_fire_samples_to_use = min(len(non_fire_start_indices), 1000)
#     else:
#         # Calculate how many non-fire samples to use to achieve the desired ratio
#         num_non_fire_samples_to_use = int((num_fire_samples / fire_ratio) - num_fire_samples)
#         num_non_fire_samples_to_use = min(num_non_fire_samples_to_use, len(non_fire_start_indices))

#     # Sample the indices to create a balanced list
#     # The crucial change is ensuring all indices are integers.
#     fire_indices_to_use = np.array(fire_start_indices, dtype=int)
#     non_fire_indices_to_use = np.random.choice(
#         non_fire_start_indices,
#         size=num_non_fire_samples_to_use,
#         replace=False
#     ).astype(int)  # Ensure non-fire indices are also integers
    
#     indices_to_use = np.concatenate([fire_indices_to_use, non_fire_indices_to_use])
#     np.random.shuffle(indices_to_use)
    
#     # --- End of sampling logic ---
    
#     for i in indices_to_use:
#         # 'i' is now guaranteed to be an integer, preventing the TypeError
#         seq_rows = df.iloc[i : i + SEQ_LEN]
#         horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        
#         X, y = build_sample(seq_rows, horizon_rows, cache)
        
#         yield X, y

In [10]:
# def make_generator(df, cache, fire_ratio=0.5):
    
#     valid_start_indices = list(range(len(df) - SEQ_LEN - HORIZONS + 1))

#     fire_start_indices = []
#     non_fire_start_indices = []
    
#     for i in valid_start_indices:
#         horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
#         has_fire = any(np.any(cache[row["viirs_file"]] > 0) for _, row in horizon_rows.iterrows())
        
#         if has_fire:
#             fire_start_indices.append(i)
#         else:
#             non_fire_start_indices.append(i)

    
#     num_fire_samples = len(fire_start_indices)
    
#     if num_fire_samples == 0:
#         print("Warning: No fire events found in the dataset. Training will be difficult.")
        
#         num_non_fire_samples_to_use = min(len(non_fire_start_indices), 1000) 
#     else:
   
#         num_non_fire_samples_to_use = int((num_fire_samples / fire_ratio) - num_fire_samples)
#         num_non_fire_samples_to_use = min(num_non_fire_samples_to_use, len(non_fire_start_indices))

#     # 4. Sample the indices to create a balanced list
#     fire_indices_to_use = fire_start_indices
#     non_fire_indices_to_use = np.random.choice(
#         non_fire_start_indices,
#         size=num_non_fire_samples_to_use,
#         replace=False 
#     )
    
#     indices_to_use = np.concatenate([fire_indices_to_use, non_fire_indices_to_use])
#     np.random.shuffle(indices_to_use)
#     indices_to_use = indices_to_use.astype(int) # Add this line to fix the type
    
#     for i in indices_to_use:
#         seq_rows = df.iloc[i : i + SEQ_LEN]
#         horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]

        
#         X, y = build_sample(seq_rows, horizon_rows, cache)
        
#         yield X, y

In [11]:
def make_generator(df, cache, fire_ratio=0.5):
    valid_start_indices = list(range(len(df) - SEQ_LEN - HORIZONS + 1))
    fire_start_indices = []
    non_fire_start_indices = []
    
    print("Scanning data for fire and non-fire events...")
    for i in valid_start_indices:
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        has_fire = any(np.any(cache[row["viirs_file"]] > 0) for _, row in horizon_rows.iterrows())
        
        if has_fire:
            fire_start_indices.append(i)
        else:
            non_fire_start_indices.append(i)

    num_fire_samples = len(fire_start_indices)
    
    if num_fire_samples == 0:
        print("Warning: No fire events found in the dataset.")
        num_non_fire_samples_to_use = min(len(non_fire_start_indices), 1000) 
    else:
        num_non_fire_samples_to_use = int((num_fire_samples / fire_ratio) - num_fire_samples)
        num_non_fire_samples_to_use = min(num_non_fire_samples_to_use, len(non_fire_start_indices))

    fire_indices_to_use = fire_start_indices

    if len(non_fire_start_indices) > 0 and num_non_fire_samples_to_use > 0:
      non_fire_indices_to_use = np.random.choice(
          non_fire_start_indices,
          size=num_non_fire_samples_to_use,
          replace=False 
      )
      indices_to_use = np.concatenate([fire_indices_to_use, non_fire_indices_to_use])
    else:
      indices_to_use = np.array(fire_indices_to_use)

    np.random.shuffle(indices_to_use)
    indices_to_use = indices_to_use.astype(int)
    
    print(f"Generator initialized. Found {len(fire_indices_to_use)} fire samples and using {len(indices_to_use) - len(fire_indices_to_use)} non-fire samples.")

    for i in indices_to_use:
        seq_rows = df.iloc[i : i + SEQ_LEN]
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        X, y = build_sample(seq_rows, horizon_rows, cache)
        yield X, y

In [12]:
# def create_dataset(df, cache, shuffle_buf=256):
#     output_signature = (
#         tf.TensorSpec(shape=(SEQ_LEN, PATCH_SIZE, PATCH_SIZE, 7), dtype=tf.float32),
#         tf.TensorSpec(shape=(HORIZONS, PATCH_SIZE, PATCH_SIZE), dtype=tf.float32),
#     )
    
#     ds = tf.data.Dataset.from_generator(
#         lambda: make_generator(df, cache),
#         output_signature=output_signature
#     )
    
#     ds = ds.shuffle(shuffle_buf, reshuffle_each_iteration=True)
#     ds = ds.prefetch(tf.data.AUTOTUNE)
    
#     return ds

In [13]:
def create_dataset(df, cache, shuffle=True, ensure_fire=True, shuffle_buf=256):
    output_signature = (
        tf.TensorSpec(shape=(SEQ_LEN, PATCH_SIZE, PATCH_SIZE, 7), dtype=tf.float32),
        tf.TensorSpec(shape=(HORIZONS, PATCH_SIZE, PATCH_SIZE), dtype=tf.float32),
    )
    
    # CORRECTED LINE:
    # Changed the keyword argument from 'ensure_fire=' to 'fire_ratio='
    # This now correctly passes the value to the make_generator function.
    ds = tf.data.Dataset.from_generator(
        lambda: make_generator(df, cache, fire_ratio=ensure_fire),
        output_signature=output_signature
    )
    
    if shuffle:
        ds = ds.shuffle(shuffle_buf, reshuffle_each_iteration=True)
    
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

In [14]:
if __name__ == "__main__":
    csv_path = r"C:\Users\Ankit\Datasets_Forest_fire\sequence_index_hourly_binary.csv"
    df = pd.read_csv(csv_path)
    
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    TOTAL = len(df)
    VAL_SPLIT = 0.2
    val_size = int(TOTAL * VAL_SPLIT)

    val_df = df.iloc[:val_size].copy()
    train_df = df.iloc[val_size:].copy()

    print(f"Total samples: {TOTAL}")
    print(f"Train samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")

    raster_cols = REQUIRED_COLS
    print("Loading rasters into memory...")
    cache = load_rasters(df, raster_cols, max_workers=8)
    print(f"Loaded {len(cache)} rasters into memory ✅")

    # Use the new, balanced dataset functions
    train_dataset = create_dataset(train_df, cache)
    val_dataset = create_dataset(val_df, cache)

Total samples: 17535
Train samples: 14028
Validation samples: 3507
Loading rasters into memory...
Loaded 9 rasters into memory ✅


In [15]:
BATCH_SIZE = 16
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [15]:
from tensorflow.keras import layers, models

In [16]:
SEQ_LEN = 6           
PATCH_H = 13            
PATCH_W = 13          
CHANNELS = 7       
HORIZONS = 3            
LSTM_UNITS = 64    
CNN_FEATURES = 128

In [9]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, callbacks

# def build_conv_lstm_unet_model(
#     seq_len=SEQ_LEN,
#     patch_h=PATCH_H,
#     patch_w=PATCH_W,
#     channels=CHANNELS,
#     horizons=HORIZONS
# ):
#     inp = layers.Input(shape=(seq_len, patch_h, patch_w, channels))

#     enc1 = layers.ConvLSTM2D(
#         filters=32, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'
#     )(inp)
#     enc1_pool = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(enc1)

#     enc2 = layers.ConvLSTM2D(
#         filters=64, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'
#     )(enc1_pool)
#     enc2_pool = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(enc2)

#     bottleneck = layers.ConvLSTM2D(
#         filters=128, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'
#     )(enc2_pool)

#     dec1_up = layers.UpSampling3D(size=(1, 2, 2))(bottleneck)
#     dec1_up = layers.Conv3D(filters=64, kernel_size=(3,3,3), padding='same', activation='relu')(dec1_up)
#     dec1_up_cropped = layers.Cropping3D(cropping=((0, 0), (0, 1), (0, 1)))(dec1_up)
#     dec1_concat = layers.Concatenate(axis=-1)([dec1_up_cropped, enc2])

#     dec2_up = layers.UpSampling3D(size=(1, 2, 2))(dec1_concat)
#     dec2_up = layers.Conv3D(filters=32, kernel_size=(3,3,3), padding='same', activation='relu')(dec2_up)
#     dec2_up_cropped = layers.Cropping3D(cropping=((0, 0), (0, 1), (0, 1)))(dec2_up)
#     dec2_concat = layers.Concatenate(axis=-1)([dec2_up_cropped, enc1])

#     output_convlstm = layers.ConvLSTM2D(
#         filters=1, kernel_size=(3, 3), padding='same', return_sequences=True, activation='sigmoid'
#     )(dec2_concat[:, :horizons])

#     final_output = tf.keras.ops.squeeze(output_convlstm, axis=-1)

#     model = models.Model(inputs=inp, outputs=final_output)
#     return model

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

def slice_output_func(x):
    return x[:, :HORIZONS, :, :, :]

def slice_output_shape(input_shape):
    return (input_shape[0], HORIZONS, input_shape[2], input_shape[3], input_shape[4])

def squeeze_output_func(x):
    return tf.squeeze(x, axis=-1)

def squeeze_output_shape(input_shape):
    return input_shape[:-1] 

def build_conv_lstm_unet_model(
    seq_len=SEQ_LEN,
    patch_h=PATCH_H,
    patch_w=PATCH_W,
    channels=CHANNELS,
    horizons=HORIZONS
):
    inp = layers.Input(shape=(seq_len, patch_h, patch_w, channels))

    enc1 = layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu')(inp)
    enc1_pool = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(enc1)

    enc2 = layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu')(enc1_pool)
    enc2_pool = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(enc2)

    bottleneck = layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu')(enc2_pool)

    dec1_up = layers.UpSampling3D(size=(1, 2, 2))(bottleneck)
    dec1_up = layers.Conv3D(filters=64, kernel_size=(3,3,3), padding='same', activation='relu')(dec1_up)
    dec1_up_cropped = layers.Cropping3D(cropping=((0, 0), (0, 1), (0, 1)))(dec1_up)
    dec1_concat = layers.Concatenate(axis=-1)([dec1_up_cropped, enc2])

    dec2_up = layers.UpSampling3D(size=(1, 2, 2))(dec1_concat)
    dec2_up = layers.Conv3D(filters=32, kernel_size=(3,3,3), padding='same', activation='relu')(dec2_up)
    dec2_up_cropped = layers.Cropping3D(cropping=((0, 0), (0, 1), (0, 1)))(dec2_up)
    dec2_concat = layers.Concatenate(axis=-1)([dec2_up_cropped, enc1])

    output_convlstm = layers.ConvLSTM2D(
        filters=1, kernel_size=(3, 3), padding='same', return_sequences=True, activation='sigmoid'
    )(dec2_concat)

    output_sliced = layers.Lambda(
        slice_output_func, 
        output_shape=slice_output_shape,
        name='output_slicer'
    )(output_convlstm)

    final_output = layers.Lambda(
        squeeze_output_func,
        output_shape=squeeze_output_shape,
        name='final_squeeze'
    )(output_sliced)
    
    model = models.Model(inputs=inp, outputs=final_output)
    return model

NameError: name 'PATCH_H' is not defined

In [18]:
from tensorflow.keras import callbacks

In [19]:
model = build_conv_lstm_unet_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 6, 13, 13, 7)      │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d (ConvLSTM2D)      │ (None, 6, 13, 13, 32)     │          45,056 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 6, 7, 7, 32)       │               0 │ conv_lstm2d[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)    │ (None, 6, 7, 7, 64)       │         221,440 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_1               │ (None, 6, 4, 4, 64)       │               0 │ conv_lstm2d_1[0][0]        │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)    │ (None, 6, 4, 4, 128)      │         885,248 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling3d (UpSampling3D)  │ (None, 6, 8, 8, 128)      │               0 │ conv_lstm2d_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 6, 8, 8, 64)       │         221,248 │ up_sampling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ cropping3d (Cropping3D)       │ (None, 6, 7, 7, 64)       │               0 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 6, 7, 7, 128)      │               0 │ cropping3d[0][0],          │
│                               │                           │                 │ conv_lstm2d_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling3d_1               │ (None, 6, 14, 14, 128)    │               0 │ concatenate[0][0]          │
│ (UpSampling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 6, 14, 14, 32)     │         110,624 │ up_sampling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ cropping3d_1 (Cropping3D)     │ (None, 6, 13, 13, 32)     │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 6, 13, 13, 64)     │               0 │ cropping3d_1[0][0],        │
│                               │                           │                 │ conv_lstm2d[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)    │ (None, 6, 13, 13, 1)      │           2,344 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 1,485,960 (5.67 MB)

 Trainable params: 1,485,960 (5.67 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()],
)

In [21]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)
checkpoint = callbacks.ModelCheckpoint(
    "best_unet_model.keras", # Changed filename
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [22]:
BATCH_SIZE = 16
steps_per_epoch = len(train_df) // BATCH_SIZE
validation_steps = len(val_df) // BATCH_SIZE

print(f"Batch size: {BATCH_SIZE}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")


Batch size: 16
Steps per epoch: 876
Validation steps: 219


In [27]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[early_stop, checkpoint],
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

Epoch 1/50
Scanning data for fire and non-fire events...
Generator initialized. Found 14020 fire samples and using 0 non-fire samples.


I0000 00:00:1758646730.610202  409982 service.cc:152] XLA service 0x7f98440068e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758646730.610219  409982 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 5060 Ti, Compute Capability 12.0
2025-09-23 22:28:50.874322: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1758646731.834813  409982 cuda_dnn.cc:529] Loaded cuDNN version 91100
I0000 00:00:1758646739.310925  409982 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


875/876 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - auc: 0.5325 - binary_accuracy: 0.9954 - loss: 0.0911Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 1: val_loss improved from None to 0.03133, saving model to best_unet_model.h5


876/876 ━━━━━━━━━━━━━━━━━━━━ 49s 35ms/step - auc: 0.5720 - binary_accuracy: 0.9952 - loss: 0.0463 - val_auc: 0.6382 - val_binary_accuracy: 0.9948 - val_loss: 0.0313
Epoch 2/50
  1/876 ━━━━━━━━━━━━━━━━━━━━ 1:12:45 5s/step - auc: 0.6670 - binary_accuracy: 0.9778 - loss: 0.1131Scanning data for fire and non-fire events...


2025-09-23 22:29:35.866010: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-09-23 22:29:35.866024: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
/usr/lib/python3.13/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - auc: 0.8134 - binary_accuracy: 0.9985 - loss: 0.0117 - val_auc: 0.6416 - val_binary_accuracy: 0.9948 - val_loss: 0.0310
Epoch 9/50
Scanning data for fire and non-fire events...
Generator initialized. Found 14020 fire samples and using 0 non-fire samples.
876/876 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - auc: 0.6381 - binary_accuracy: 0.9951 - loss: 0.0295Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 9: val_loss improved from 0.03102 to 0.03095, saving model to best_unet_model.h5


876/876 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - auc: 0.6386 - binary_accuracy: 0.9952 - loss: 0.0292 - val_auc: 0.6609 - val_binary_accuracy: 0.9948 - val_loss: 0.0310
Epoch 10/50
  1/876 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - auc: 0.6794 - binary_accuracy: 0.9990 - loss: 0.0100Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 10: val_loss did not improve from 0.03095
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - auc: 0.6794 - binary_accuracy: 0.9990 - loss: 0.0100 - val_auc: 0.6525 - val_binary_accuracy: 0.9948 - val_loss: 0.0310
Epoch 11/50
Scanning data for fire and non-fire events...
Generator initialized. Found 14020 fire samples and using 0 non-fire samples.
876/876 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - auc: 0.6441 - binary_accuracy: 0.9955 - loss: 0.0276Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 11: val_loss did not improve 

876/876 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - auc: 0.6511 - binary_accuracy: 0.9952 - loss: 0.0291 - val_auc: 0.6732 - val_binary_accuracy: 0.9948 - val_loss: 0.0309
Epoch 14/50
  1/876 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - auc: 0.6392 - binary_accuracy: 0.9990 - loss: 0.0104Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 14: val_loss did not improve from 0.03090
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - auc: 0.6392 - binary_accuracy: 0.9990 - loss: 0.0104 - val_auc: 0.6739 - val_binary_accuracy: 0.9948 - val_loss: 0.0309
Epoch 15/50
Scanning data for fire and non-fire events...
Generator initialized. Found 14020 fire samples and using 0 non-fire samples.
875/876 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - auc: 0.6552 - binary_accuracy: 0.9953 - loss: 0.0284Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 15: val_loss did not improve 

2025-09-23 22:33:44.002168: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2025-09-23 22:33:44.002192: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 2109665068740041596


Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 16: val_loss did not improve from 0.03090
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - auc: 0.0000e+00 - binary_accuracy: 1.0000 - loss: 0.0038 - val_auc: 0.6127 - val_binary_accuracy: 0.9948 - val_loss: 0.0314
Epoch 17/50
Scanning data for fire and non-fire events...
Generator initialized. Found 14020 fire samples and using 0 non-fire samples.
874/876 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - auc: 0.6493 - binary_accuracy: 0.9952 - loss: 0.0291Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.

Epoch 17: val_loss did not improve from 0.03090
876/876 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - auc: 0.6548 - binary_accuracy: 0.9952 - loss: 0.0290 - val_auc: 0.6326 - val_binary_accuracy: 0.9948 - val_loss: 0.0311
Epoch 18/50
  1/876 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - auc: 0.0000e+00 - binary_accuracy: 1.0000 - loss: 0.0040Scanning data for fire an

In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from sklearn.metrics import classification_report # Requires: pip install scikit-learn

# --- 1. Define Global Variables (CRITICAL) ---
# These values are needed by the custom functions and to shape the targets.
SEQ_LEN = 6
HORIZONS = 3
PATCH_H = 13
PATCH_W = 13
CHANNELS = 7

# --- 2. Define Custom Functions (CRITICAL for Model Loading) ---
# These functions must be present in the namespace.

def slice_output_func(x):
    return x[:, :HORIZONS, :, :, :] 

def squeeze_output_func(x):
    return tf.squeeze(x, axis=-1)

# Define shape functions (needed by the H5 format model)
def slice_output_shape(input_shape):
    return (input_shape[0], HORIZONS, input_shape[2], input_shape[3], input_shape[4])

def squeeze_output_shape(input_shape):
    return input_shape[:-1] 

CUSTOM_OBJECTS = {
    'slice_output_func': slice_output_func,
    'slice_output_shape': slice_output_shape,
    'squeeze_output_func': squeeze_output_func,
    'squeeze_output_shape': squeeze_output_shape,
    'output_slicer': layers.Lambda(slice_output_func, output_shape=slice_output_shape, name='output_slicer'),
    'final_squeeze': layers.Lambda(squeeze_output_func, output_shape=squeeze_output_shape, name='final_squeeze'),
} 

MODEL_PATH = r"C:\Users\Ankit\Downloads\best_unet_model (2).h5" 

# --- 3. Load the Model ---
try:
    model = tf.keras.models.load_model(
        MODEL_PATH, 
        custom_objects=CUSTOM_OBJECTS,
        safe_mode=False # Still needed for legacy H5 Lambda layers
    )
    print("Model loaded successfully!")
except Exception as e:
    print(f"\n[ERROR] Failed to load model: {e}")
    exit()

# --- 4. Prediction and Report Generation ---

# Note: The 'val_dataset' must be defined and available from your prior data loading code.
if 'val_dataset' not in locals():
    print("\n[CRITICAL WARNING] 'val_dataset' is not available. Generating a report requires your validation data.")
    print("Please ensure your data loading cells from the original notebook are executed.")
    # For demonstration, we will use a small prediction sample if available
    # Assuming you have the val_dataset iterator from your previous steps:
    # X_val, y_val = next(iter(val_dataset)) # Uncomment this if you can access your dataset
    exit()


print("\n--- Generating Classification Report ---")

# a. Run predictions on the full validation dataset
# Setting verbose=0 silences the progress bar
y_pred_probs = model.predict(val_dataset, verbose=0) 

# b. Get true labels. We need to iterate through the dataset to get all labels.
y_true_list = []
for _, y_batch in val_dataset:
    y_true_list.append(y_batch.numpy())

y_true = np.concatenate(y_true_list, axis=0)

# c. Flatten the arrays and convert probabilities to binary labels (0 or 1)
# The prediction shape is (N_samples, HORIZONS, H, W). We flatten it to a single list of pixels.
y_true_flat = y_true.flatten()

# Use a threshold (0.5 is standard) to convert probabilities to classes
y_pred_classes_flat = (y_pred_probs.flatten() > 0.5).astype(int)

# d. Generate and print the classification report
report = classification_report(y_true_flat, y_pred_classes_flat, target_names=['No Fire (0)', 'Fire (1)'])

print(report)

print("--- End of Report ---")

Model loaded successfully!

[CRITICAL WARNING] 'val_dataset' is not available. Generating a report requires your validation data.
Please ensure your data loading cells from the original notebook are executed.

--- Generating Classification Report ---


NameError: name 'val_dataset' is not defined

In [2]:
import tensorflow as tf

In [2]:
import tensorflow as tf

# NOTE: These functions must be defined, as your model structure requires them.
def slice_output_func(x):
    return x[:, :3, :, :, :] 

def squeeze_output_func(x):
    return tf.squeeze(x, axis=-1)

CUSTOM_OBJECTS = {
    'output_slicer': slice_output_func,
    'final_squeeze': squeeze_output_func,
} 

# Update this path if you used a temporary location like C:\Temp
MODEL_PATH = r"C:\Users\Ankit\best_unet_model.keras"

try:
    print("Attempting to load model after system reboot...")
    model = tf.keras.models.load_model(
        MODEL_PATH, 
        custom_objects=CUSTOM_OBJECTS
    )
    
    print("\nModel loaded successfully! The system lock was successfully cleared.")
    model.summary()
    
except Exception as e:
    # If this fails after a clean reboot, there is a fundamental (but rare) 
    # operating system permission issue you need to fix manually.
    print(f"\n[CRITICAL ERROR] Failed to load model even after reboot: {e}")

Attempting to load model after system reboot...

[CRITICAL ERROR] Failed to load model even after reboot: [Errno 13] Permission denied: 'C:\\Users\\Ankit\\best_unet_model.keras'


In [4]:
import torch
import sys

def check_pytorch_gpu():
    """
    Checks for GPU availability in PyTorch, prints details, and runs a test.
    """
    print(f"--- PyTorch and System Information ---")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"Python Version: {sys.version}")

    # The primary function to check for a CUDA-enabled GPU
    is_available = torch.cuda.is_available()

    print(f"\n--- GPU Detection ---")
    if is_available:
        # Get the number of available GPUs
        gpu_count = torch.cuda.device_count()
        print(f"✅ Success! PyTorch has detected {gpu_count} CUDA-enabled GPU(s).")

        # Print details for each GPU
        for i in range(gpu_count):
            gpu_name = torch.cuda.get_device_name(i)
            current_device_index = torch.cuda.current_device()
            # Add a star '*' to indicate the currently active GPU
            active_indicator = " (Active)" if i == current_device_index else ""
            print(f"  GPU [{i}]: {gpu_name}{active_indicator}")

        print("\n--- Simple GPU Operation Test ---")
        try: # 1. Define the device to be the first available GPU
            device = torch.device("cuda:0")
            print(f"  Attempting to use device: {device} ({torch.cuda.get_device_name(0)})")

            # 2. Create a sample tensor and move it to the GPU
            sample_tensor = torch.tensor([1.5, 2.5, 3.5], device=device)
            print(f"  Successfully created a tensor on the GPU.")
            print(f"  Tensor: {sample_tensor}")
            print(f"  Tensor's Device: {sample_tensor.device}")

            # 3. Perform a simple operation
            result = sample_tensor * 2
            print("\n  Performing a simple operation (tensor * 2)...")
            print(f"  Result: {result}")
            print(f"  Result's Device: {result.device}")
            print("\n✅ GPU is working correctly for computations.")

        except Exception as e:
            print(f"❌ An error occurred during the GPU operation test: {e}")

    else:
        print("❌ Failure. PyTorch did NOT detect any CUDA-enabled GPUs.")
        print("Please check the following:")
        print("1. Is a compatible NVIDIA GPU installed?")
        print("2. Are the NVIDIA drivers installed correctly? (Check with 'nvidia-smi' in your terminal)")
        print("3. Did you install the PyTorch version with CUDA support?")
        print("   (e.g., from pytorch.org, select a CUDA option in the install matrix)")

if __name__ == "__main__":
    check_pytorch_gpu()

--- PyTorch and System Information ---
PyTorch Version: 2.8.0
Python Version: 3.13.7 (main, Aug 15 2025, 12:34:02) [GCC 15.2.1 20250813]

--- GPU Detection ---
✅ Success! PyTorch has detected 1 CUDA-enabled GPU(s).
  GPU [0]: NVIDIA GeForce RTX 5060 Ti (Active)

--- Simple GPU Operation Test ---
  Attempting to use device: cuda:0 (NVIDIA GeForce RTX 5060 Ti)
  Successfully created a tensor on the GPU.
  Tensor: tensor([1.5000, 2.5000, 3.5000], device='cuda:0')
  Tensor's Device: cuda:0

  Performing a simple operation (tensor * 2)...
  Result: tensor([3., 5., 7.], device='cuda:0')
  Result's Device: cuda:0

✅ GPU is working correctly for computations.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from datetime import datetime, timedelta
import os
import sys

# --- 1. Define Global Variables (CRITICAL) ---
SEQ_LEN = 6      # Input sequence length (days/timesteps)
HORIZONS = 3     # Prediction horizon length (days/timesteps)
PATCH_H = 13     # Patch height
PATCH_W = 13     # Patch width
CHANNELS = 7
BATCH_SIZE = 16

# --- 2. Define Custom Functions (Required for Model Loading) ---
# NOTE: These functions must be in the global scope.
def slice_output_func(x):
    return x[:, :HORIZONS, :, :, :] 

def squeeze_output_func(x):
    return tf.squeeze(x, axis=-1)

def slice_output_shape(input_shape):
    return (input_shape[0], HORIZONS, input_shape[2], input_shape[3], input_shape[4])

def squeeze_output_shape(input_shape):
    return input_shape[:-1] 

CUSTOM_OBJECTS = {
    'slice_output_func': slice_output_func,
    'slice_output_shape': slice_output_shape,
    'squeeze_output_func': squeeze_output_func,
    'squeeze_output_shape': squeeze_output_shape,
    'output_slicer': layers.Lambda(slice_output_func, output_shape=slice_output_shape, name='output_slicer'),
    'final_squeeze': layers.Lambda(squeeze_output_func, output_shape=squeeze_output_shape, name='final_squeeze'),
} 

# --- 3. Model Loading ---
MODEL_PATH = r"C:\Users\Ankit\Downloads\final_model.h5"
try:
    print(f"Loading model from: {MODEL_PATH}")
    model = tf.keras.models.load_model(
        MODEL_PATH, 
        custom_objects=CUSTOM_OBJECTS,
        safe_mode=False 
    )
    print("Model loaded successfully.")
except Exception as e:
    print(f"\n[CRITICAL ERROR] Failed to load model: {e}")
    sys.exit(1) # Exit if model load fails


# --- 4. Define Helper Functions for Date/Time Mapping ---

def get_sample_date_range(start_index, df):
    """Calculates the date range for a given sample index."""
    # The start date for the dataset (January 1, 2015)
    START_DATE = datetime(2015, 1, 1)
    
    # Each row in your sequence_index_hourly_binary.csv represents one hour (based on context).
    # The 'start_index' corresponds to the index in the original DataFrame (df).
    
    # The prediction starts at the hour corresponding to the end of the input sequence.
    pred_start_index = start_index + SEQ_LEN
    pred_end_index = pred_start_index + HORIZONS - 1
    
    # Calculate the datetime objects
    pred_start_time = START_DATE + timedelta(hours=int(df.iloc[pred_start_index]['index']))
    pred_end_time = START_DATE + timedelta(hours=int(df.iloc[pred_end_index]['index']))
    
    return pred_start_time.strftime('%Y-%m-%d %H:%M:%S'), pred_end_time.strftime('%Y-%m-%d %H:%M:%S')


# --- 5. Find Successful Fire Prediction ---

# --- CRITICAL ASSUMPTIONS ---
# 1. 'train_df' and 'val_df' (DataFrames used for creating the datasets) are available.
# 2. 'val_dataset' (TensorFlow Dataset iterator) is available and correctly setup.
# 3. 'df' (the original, unshuffled, full DataFrame) is available if you want to map back to original indices.

# Since we can't rely on global variables in this environment, we'll try to find the starting index 
# and report the prediction. The logic is self-contained below.

# --- Finding the Correct Prediction ---
FIRE_PREDICTION_FOUND = False
THRESHOLD = 0.5
sample_counter = 0

print("\n--- Searching for a Correct Fire Prediction Sample ---")
print(f"Using prediction threshold: {THRESHOLD}")


# We need to recreate a simplified generator/iterator logic here if we can't access val_dataset.
# ASSUMING: You have already run the data loading cells and have the actual 'val_dataset'
# If not, this script WILL FAIL at the 'val_dataset' iteration.

for X_batch, Y_true_batch in val_dataset.unbatch().batch(1): # Iterate one sample at a time
    
    # 1. Get the prediction
    Y_pred_batch = model.predict(X_batch, verbose=0)
    
    # 2. Convert prediction probabilities to binary class (0 or 1)
    Y_pred_classes = (Y_pred_batch > THRESHOLD).astype(int)
    
    # 3. Check for a True Positive (TP): Ground Truth is Fire (1) AND Prediction is Fire (1)
    # The true fire samples are where Y_true_batch > 0 
    
    # Check if the ground truth contains *any* fire pixel
    is_fire_in_true = np.any(Y_true_batch.numpy() > 0)
    
    if is_fire_in_true:
        # Check if the model correctly predicted fire for *at least one* fire pixel
        fire_pixels_correctly_predicted = np.sum(
            (Y_true_batch.numpy() > 0) & (Y_pred_classes > 0)
        )
        
        if fire_pixels_correctly_predicted > 0:
            FIRE_PREDICTION_FOUND = True
            
            # Since we cannot access the original index from the generator efficiently, 
            # we report the sample counter as a demonstration index.
            
            print("\n✅ **SUCCESS! TRUE POSITIVE SAMPLE FOUND.**")
            print(f"Local Sample Index in Validation Dataset: {sample_counter}")
            print(f"Total True Fire Pixels in Sample: {np.sum(Y_true_batch.numpy() > 0)}")
            print(f"Correctly Predicted Fire Pixels (TP): {fire_pixels_correctly_predicted}")
            
            
            # --- Date/Time Output ---
            # To get the real date, you must map this sample_counter back to the original index in val_df.
            if 'val_df' in locals():
                try:
                    # The index in val_df corresponding to this sample (approximation)
                    # NOTE: This is an approximation since generator shuffling complicates index mapping.
                    approx_df_index = val_df.index[sample_counter]
                    
                    # Assuming df (full original dataframe) is also available and indexed by the original csv rows.
                    full_df_index = df.index[approx_df_index] # Get the original index from the full df
                    
                    start_time, end_time = get_sample_date_range(full_df_index, df)
                    print(f"\n--- Prediction Window (APPROXIMATION) ---")
                    print(f"Input Sequence Start: {start_time}")
                    print(f"Prediction Horizon End: {end_time}")
                except Exception as e:
                    print(f"Could not map to date range. Error: {e}")
                    
            # Stop after finding the first one
            break
            
    sample_counter += 1

if not FIRE_PREDICTION_FOUND:
    print("\n❌ No True Positive fire predictions found in the first pass of the validation dataset.")
    print("Your model may be biased towards predicting 'No Fire' even for high-AUC samples.")

Loading model from: C:\Users\Ankit\Downloads\final_model.h5


Model loaded successfully.

--- Searching for a Correct Fire Prediction Sample ---
Using prediction threshold: 0.5
Scanning data for fire and non-fire events...
Generator initialized. Found 3499 fire samples and using 0 non-fire samples.
